In [ ]:
# Import libraries
from IPython.display import display
import ipywidgets as ipw
import utils

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

openbis_connection_status_htmlbox = utils.HTMLbox(value = '')
if OPENBIS_SESSION:
    openbis_connection_status_htmlbox.value = CONFIG["home_page"]["enable_status"]
    open_notebooks_html_enable_code = ''.join(CONFIG["home_page"]["enable_links"])
    open_notebooks_htmlbox = utils.HTMLbox(value = open_notebooks_html_enable_code)
else:
    openbis_connection_status_htmlbox.value = CONFIG["home_page"]["disable_status"]
    open_notebooks_html_disable_code = ''.join(CONFIG["home_page"]["disable_links"])
    open_notebooks_htmlbox = utils.HTMLbox(value = open_notebooks_html_disable_code)

# AiiDAlab-openBIS app

In [ ]:
display(openbis_connection_status_htmlbox)

## Create and/or edit objects in openBIS

In [ ]:
display(ipw.VBox([open_notebooks_htmlbox], layout = ipw.Layout(width='100%', border='solid 1px black')))

In [13]:
%load_ext aiida
%aiida
from aiida import orm
qb = orm.QueryBuilder()
qb.append(orm.WorkChainNode)
results = qb.all()
x = results[0]
y = x[0]
y.process_label

The aiida extension is already loaded. To reload it, use:
  %reload_ext aiida


'Cp2kBaseWorkChain'